<a href="https://colab.research.google.com/github/guymatz/csci-49376/blob/develop/csci_49376_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
#drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
!pip install pyspark

In [3]:
# Import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql import functions
# Create a Spark Session
ss = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
edges = ss.read.csv("/content/drive/MyDrive/School/CSCI/Big-data/edges.tsv", header=True, sep="\t")
nodes = ss.read.csv("/content/drive/MyDrive/School/CSCI/Big-data/nodes.tsv", header=True, sep="\t")

In [5]:
print(f"Edges: {edges.count()}")
print(f"Nodes: {nodes.count()}")

Edges: 1292203
Nodes: 23036


In [6]:
# 1. For each drug, compute the number of genes
# and the number of diseases associated with the
# drug. Output results with top 5 number of genes in a
# descending order

In [7]:
compounds = edges.filter(
    edges.source.startswith('Compound') &
    (
        edges.target.startswith('Gene') 
        |
        edges.target.startswith('Disease')
    )
)

In [55]:
compounds.sample(.01).show(10)

+-----------------+--------+-------------------+
|           source|metaedge|             target|
+-----------------+--------+-------------------+
|Compound::DB00783|     CtD|Disease::DOID:10283|
|Compound::DB00222|     CtD| Disease::DOID:9352|
|Compound::DB00860|     CtD| Disease::DOID:2531|
|Compound::DB00394|     CtD| Disease::DOID:8893|
|Compound::DB01005|     CtD| Disease::DOID:2531|
|Compound::DB01234|     CtD| Disease::DOID:7148|
|Compound::DB00230|     CtD| Disease::DOID:1826|
|Compound::DB00635|     CtD| Disease::DOID:9008|
|Compound::DB01186|     CbG|          Gene::148|
|Compound::DB00839|     CbG|         Gene::3767|
+-----------------+--------+-------------------+
only showing top 10 rows



In [69]:
c3 = compounds.withColumn("target_type", functions.split(compounds.target, '::').getItem(0))
c4 = c3.drop('metaedge', 'target')
c4.sort('source').show(5)

+-----------------+-----------+
|           source|target_type|
+-----------------+-----------+
|Compound::DB00014|    Disease|
|Compound::DB00014|    Disease|
|Compound::DB00014|       Gene|
|Compound::DB00014|       Gene|
|Compound::DB00014|       Gene|
+-----------------+-----------+
only showing top 5 rows



In [70]:
c4.groupBy('source','target_type').count().sort('count', 'source', ascending=False).show()

+-----------------+-----------+-----+
|           source|target_type|count|
+-----------------+-----------+-----+
|Compound::DB08865|       Gene|  585|
|Compound::DB01254|       Gene|  564|
|Compound::DB00997|       Gene|  532|
|Compound::DB00570|       Gene|  523|
|Compound::DB00390|       Gene|  522|
|Compound::DB00170|       Gene|  521|
|Compound::DB00694|       Gene|  519|
|Compound::DB00679|       Gene|  519|
|Compound::DB00188|       Gene|  517|
|Compound::DB00441|       Gene|  515|
|Compound::DB08912|       Gene|  514|
|Compound::DB00091|       Gene|  513|
|Compound::DB01204|       Gene|  511|
|Compound::DB00877|       Gene|  511|
|Compound::DB02546|       Gene|  510|
|Compound::DB01092|       Gene|  509|
|Compound::DB00444|       Gene|  509|
|Compound::DB01103|       Gene|  508|
|Compound::DB00445|       Gene|  508|
|Compound::DB01396|       Gene|  507|
+-----------------+-----------+-----+
only showing top 20 rows



In [ ]:
# Q2: Compute the number of diseases associated
# with 1, 2, 3, …, n drugs. Output results with the top
# 5 number of diseases in a descending order.
disease_compounds = edges.filter(
    edges.source.startswith('Compound') &
    (
        edges.target.startswith('Disease')
    )

)

In [ ]:
filtered_disease_compounds = disease_compounds.rdd.map(lambda x: (x.target,1))
filtered_disease_compounds.take(5)
filtered_disease_compound_counts = filtered_disease_compounds.reduceByKey(lambda a,b:a + b)
sorted_disease_count = filtered_disease_compound_counts.sortBy(lambda x: x[1], ascending=False)
sorted_disease_count.take(5)

[('Disease::DOID:10763', 70),
 ('Disease::DOID:2531', 53),
 ('Disease::DOID:1612', 43),
 ('Disease::DOID:2841', 40),
 ('Disease::DOID:3393', 38)]

In [ ]:
# Q3: Get the name of drugs that have the top 5 number of genes. Output the results.